## Установка зависимостей и загрузка библиотек

Устанавливаем зависимости и подключаем библиотеки для RAG-Telegram-бота: векторный поиск и хранилище (Qdrant), эмбеддинги текста (Sentence Transformers), загрузка и чанкинг документов (LlamaIndex), генерация ответов LLM (OpenAI API) и интерфейс Telegram-бота (pyTelegramBotAPI).

In [1]:
!pip install -q qdrant-client==1.16.2 sentence-transformers llama-index openai
!pip install -q pyTelegramBotAPI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [19]:
import os
import json
import re
import numpy as np
import requests
import telebot
import threading
from typing import List, Dict, Any

from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from openai import OpenAI

# Конфигурация из config.py (секреты из .env или переменных окружения; в Colab — через Secrets)
import config
openai_client = OpenAI(api_key=config.OPENAI_API_KEY)

## Инициализация Qdrant и конфигурация

Код настраивает подключение к Qdrant и проверяет наличие векторной коллекции. Если коллекция не существует, он создаёт её с заданной размерностью векторов и косинусной метрикой, подготавливая хранилище для эмбеддингов и последующего векторного поиска.

In [23]:
# Конфигурация из config
QDRANT_URL = config.QDRANT_URL
QDRANT_API_KEY = config.QDRANT_API_KEY
collection_name = config.COLLECTION_NAME
embed_model_name = config.EMBED_MODEL_NAME
vector_size = config.VECTOR_SIZE
# Qdrant клиент
qdrant = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY
)

# Создание коллекции если не существует
existing = [c.name for c in qdrant.get_collections().collections]
if collection_name not in existing:
    qdrant.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(size=vector_size, distance=models.Distance.COSINE),
    )
print("Collection:", collection_name)

Collection: career_levels_bge_v2


## Загрузка и обработка документа

- Загрузка и обработка данных
- Загрузка документа text2.txt
- Разделение текста на чанки (chunk_size=256, overlap=80)
- Подготовка текстовых фрагментов для индексации

In [4]:
# Загрузка документа
documents = SimpleDirectoryReader(input_files=["text2.txt"]).load_data()
print("Loaded documents:", len(documents))
print(documents[0].text[:300])

# Чанкинг
splitter = SentenceSplitter(chunk_size=config.CHUNK_SIZE, chunk_overlap=config.CHUNK_OVERLAP)
nodes = splitter.get_nodes_from_documents(documents)
texts = [n.text for n in nodes]
print("Chunks (nodes):", len(texts))
print(texts[0][:300])

Loaded documents: 1
Бэкенд-разработчик отвечает за серверную часть продукта: бизнес-логику, работу с данными, интеграции, безопасность и надежность систем. Его работа обеспечивает стабильность, масштабируемость и предсказуемость пользовательского опыта, даже если конечный пользователь напря
Chunks (nodes): 56
Бэкенд-разработчик отвечает за серверную часть продукта: бизнес-логику, работу с данными, интеграции, безопасность и надежность систем. Его работа обеспечивает стабильность, масштабируемость и предсказуемость пользовательского опыта, даже если конечный пользователь напря


## Создание эмбеддингов и индексация

Код преобразует текстовые чанки в эмбеддинги с помощью модели SentenceTransformer, нормализуя векторы для корректного косинусного поиска. Затем для каждого чанка формируется точка с вектором и метаданными. Эти точки загружаются в Qdrant с помощью upsert, создавая или обновляя векторный индекс. В результате данные становятся доступными для быстрого семантического поиска и использования в RAG.

In [5]:
# Эмбеддинги
embedder = SentenceTransformer(embed_model_name)

vectors = embedder.encode(
    texts,
    normalize_embeddings=True,
    batch_size=64,
    show_progress_bar=True,
)

# Подготовка точек для Qdrant
points = [
    models.PointStruct(
        id=idx,
        vector=vectors[idx].tolist(),
        payload={"text": texts[idx], "chunk_id": idx, "source": "text.txt"},
    )
    for idx in range(len(texts))
]

# Загрузка в Qdrant
qdrant.upsert(collection_name=collection_name, points=points)
print("Upserted:", len(points))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Upserted: 56


## Функции ретривера и генерации ответов

Сначала функция retrieve преобразует вопрос в эмбеддинг и выполняет векторный поиск в Qdrant, возвращая наиболее релевантные текстовые чанки. Затем generate_rag_answer собирает найденные чанки в контекст и передаёт его в LLM (OpenAI), которая формирует ответ, строго опираясь на этот контекст. Функция answer объединяет оба шага и возвращает пользователю финальный ответ либо сообщение об отсутствии релевантных знаний в базе.

In [ ]:
def retrieve(query: str, top_k: int = 10):
    """Функция поиска релевантных чанков"""
    qvec = embedder.encode([query], normalize_embeddings=True)[0].tolist()

    headers = {"Content-Type": "application/json"}
    if QDRANT_API_KEY:
        headers["api-key"] = QDRANT_API_KEY

    payload = {
        "vector": qvec,
        "limit": top_k,
        "with_payload": True,
        "with_vector": False,
    }

    r = requests.post(
        f"{QDRANT_URL}/collections/{collection_name}/points/search",
        json=payload,
        headers=headers,
        timeout=60,
    )
    r.raise_for_status()
    return r.json()["result"]

hits = retrieve("Какие навыки нужно развивать, чтобы стать бэкенд разработчиком?", top_k=10)

for i, h in enumerate(hits, 1):
    print(f"\n#{i} score={h['score']:.4f} chunk_id={h['payload'].get('chunk_id')}")
    print(h["payload"]["text"][:300])


def generate_rag_answer(question: str, hits, model: str = "gpt-4o-mini") -> str:
    """Генерация ответа на основе найденных чанков"""
    context = "\n\n".join(
        f"[chunk {h['payload'].get('chunk_id')}] {h['payload']['text']}"
        for h in hits
    )

    rag_prompt = f"""Ты карьерный ассистент. Отвечай на вопрос, опираясь на контекст ниже. Выдавай 5-7 навыков из контекста и давай к ним пояснение строго из контекста.
Если в контексте нет ответа — честно скажи, что информации недостаточно и предложи 2–3 уточняющих вопроса.

Контекст:
{context}

Вопрос:
{question}
"""

    resp = openai_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": rag_prompt}],
        stream=False,
    )
    return resp.choices[0].message.content

def answer(question: str, top_k: int = None, model: str = None, score_threshold: float = None) -> str:
    """Основная функция для получения ответа. Чанки с score ниже score_threshold не передаются в LLM."""
    top_k = top_k if top_k is not None else config.TOP_K
    model = model or config.LLM_MODEL
    score_threshold = score_threshold if score_threshold is not None else config.SCORE_THRESHOLD
    hits = retrieve(question, top_k=top_k)
    if score_threshold > 0:
        hits = [h for h in hits if h.get("score", 0) >= score_threshold]
    if not hits:
        return "Я не нашёл релевантных знаний в базе. Попробуй переформулировать вопрос."
    return generate_rag_answer(question, hits, model)

## Телеграм бот

In [ ]:
bot = telebot.TeleBot(config.TELEGRAM_BOT_TOKEN)
MAX_LEN = config.MAX_MESSAGE_LEN
from rag_logger import setup_bot_logger, log_request, log_error
bot_logger = setup_bot_logger(config.LOG_DIR)  

def split_message(text, max_len=MAX_LEN):
    return [text[i:i+max_len] for i in range(0, len(text), max_len)] if text else [""]

@bot.message_handler(commands=["start", "help"])
def start_help(message):
    bot.reply_to(
        message,
        "Привет! Напиши вопрос — я отвечу с опорой на базу.\n",
        parse_mode='Markdown'
    )

@bot.message_handler(func=lambda m: True, content_types=["text"])
def handle_text(message):
    q = (message.text or "").strip()
    if not q:
        return
    try:
        bot.send_chat_action(message.chat.id, "typing")
        resp = answer(q)
        log_request(bot_logger, q, len(resp))
        for part in split_message(resp):
            bot.reply_to(message, part, parse_mode='Markdown')
    except Exception as e:
        log_error(bot_logger, q, e)
        bot.reply_to(message, f"Ошибка: {e}", parse_mode='Markdown')

def run_bot():
    bot.infinity_polling(skip_pending=True, timeout=60, long_polling_timeout=60)

# Запуск бота в отдельном потоке
threading.Thread(target=run_bot, daemon=True).start()
print("Тг бот активен")

Тг бот активен


## Функции для оценки качества RAG

In [8]:
# Функции для расчета метрик качества
def simple_text_similarity(text1: str, text2: str) -> float:
    """Простая оценка схожести текстов для проверки релевантности"""
    text1_clean = re.sub(r'\s+', ' ', text1.lower()).strip()
    text2_clean = re.sub(r'\s+', ' ', text2.lower()).strip()

    if text1_clean in text2_clean or text2_clean in text1_clean:
        return 1.0

    words1 = set(text1_clean.split())
    words2 = set(text2_clean.split())

    if not words1 or not words2:
        return 0.0

    intersection = words1.intersection(words2)
    return len(intersection) / max(len(words1), len(words2))

def calculate_context_precision(retrieved_contexts: List[str],
                                relevant_contexts: List[str],
                                top_k: int = 5) -> float:
    """Вычисляет Context Precision"""
    if not retrieved_contexts:
        return 0.0

    k = min(top_k, len(retrieved_contexts))
    retrieved = retrieved_contexts[:k]

    relevant_retrieved = 0
    precision_at_k_sum = 0

    for i in range(1, k + 1):
        current_context = retrieved[i-1]
        is_relevant = False

        for ref_context in relevant_contexts:
            if simple_text_similarity(current_context, ref_context) > 0.3:
                is_relevant = True
                break

        if is_relevant:
            relevant_retrieved += 1

        precision_at_i = relevant_retrieved / i

        if is_relevant:
            precision_at_k_sum += precision_at_i

    if relevant_retrieved == 0:
        return 0.0

    context_precision = precision_at_k_sum / relevant_retrieved
    return min(1.0, context_precision)

def calculate_context_recall(retrieved_contexts: List[str],
                            reference_answer: str) -> float:
    """Упрощенный расчет Context Recall"""
    statements = []
    for sentence in re.split(r'[.,;]', reference_answer):
        sentence = sentence.strip()
        if len(sentence) > 20:
            statements.append(sentence)

    if not statements:
        return 0.0

    supported_statements = 0

    for statement in statements:
        statement_norm = statement.lower()
        statement_words = set(re.findall(r'\w+', statement_norm))

        statement_supported = False

        for context in retrieved_contexts:
            context_norm = context.lower()
            context_words = set(re.findall(r'\w+', context_norm))

            common_words = statement_words.intersection(context_words)
            if len(common_words) >= 3:
                statement_supported = True
                break

        if statement_supported:
            supported_statements += 1

    context_recall = supported_statements / len(statements)
    return min(1.0, context_recall)

## Запуск оценки качества

In [24]:
import requests

r = requests.get(
    f"{QDRANT_URL.rstrip('/')}/collections",
    headers={"api-key": QDRANT_API_KEY}
)
print(r.json())

{'result': {'collections': [{'name': 'knowledge_base'}, {'name': 'career_levels_hybrid_v3'}, {'name': 'career_levels'}, {'name': 'career_levels_improved_v1'}, {'name': 'career_levels_bge_v1'}, {'name': 'career_levels_bge_v2'}, {'name': 'career_kb_hybrid'}]}, 'status': 'ok', 'time': 1.248e-05}


In [27]:
def evaluate_rag_system(test_data: List[Dict], top_k: int = 5) -> Dict[str, Any]:
    """Полная оценка RAG-системы"""
    all_precisions = []
    all_recalls = []
    detailed_results = []

    for i, item in enumerate(test_data):
        question = item["question"]
        reference_answer = item["reference_answer"]
        reference_contexts = item["reference_contexts"]

        hits = retrieve(question, top_k=top_k)

        if not hits:
            rag_answer = ""
            retrieved_contexts = []
        else:
            rag_answer = generate_rag_answer(question, hits)
            retrieved_contexts = [h["payload"]["text"] for h in hits]

        precision = calculate_context_precision(retrieved_contexts, reference_contexts, top_k)
        recall = calculate_context_recall(retrieved_contexts, reference_answer)

        all_precisions.append(precision)
        all_recalls.append(recall)

        detailed_results.append({
            "question": question,
            "rag_answer": rag_answer[:200] + "..." if len(rag_answer) > 200 else rag_answer,
            "reference_answer": reference_answer[:200] + "..." if len(reference_answer) > 200 else reference_answer,
            "retrieved_contexts_count": len(retrieved_contexts),
            "relevant_contexts_count": len(reference_contexts),
            "context_precision": precision,
            "context_recall": recall,
            "retrieved_contexts": retrieved_contexts[:2]
        })

    avg_precision = np.mean(all_precisions) if all_precisions else 0.0
    avg_recall = np.mean(all_recalls) if all_recalls else 0.0

    return {"context_precision": avg_precision,
        "context_recall": avg_recall,
        "num_questions": len(test_data),
        "detailed_results": detailed_results}

def run_evaluation():
    """Запуск оценки RAG-системы"""
    with open('test_dataset.json', 'r', encoding='utf-8') as f:
        test_qa_pairs = json.load(f)

    results = evaluate_rag_system(test_qa_pairs, top_k=5)

    print(f"Context Precision (средняя): {results['context_precision']:.4f}")
    print(f"Context Recall (средняя):    {results['context_recall']:.4f}")
    print(f"Оценено вопросов:           {results['num_questions']}")
    print("-"*60)
    #посмотрим по каждому вопросу
    for i, detail in enumerate(results['detailed_results']):
        print(f"\n{i+1}. Вопрос: {detail['question'][:80]}...")
        print(f"   Context Precision: {detail['context_precision']:.3f}")
        print(f"   Context Recall:    {detail['context_recall']:.3f}")
        print(f"   Извлечено контекстов: {detail['retrieved_contexts_count']}")
        print(f"   Релевантных контекстов: {detail['relevant_contexts_count']}")

    return results

# Запуск оценки
if __name__ == "__main__":
    run_evaluation()

Context Precision (средняя): 0.6467
Context Recall (средняя):    0.6080
Оценено вопросов:           10
------------------------------------------------------------

1. Вопрос: Какие навыки нужно развивать, чтобы стать бэкенд-разработчиком?...
   Context Precision: 0.950
   Context Recall:    0.111
   Извлечено контекстов: 5
   Релевантных контекстов: 10

2. Вопрос: Что входит в обязанности фронтенд-разработчика?...
   Context Precision: 0.833
   Context Recall:    0.500
   Извлечено контекстов: 5
   Релевантных контекстов: 6

3. Вопрос: Что должен знать и уметь аналитик-разработчик?...
   Context Precision: 0.589
   Context Recall:    0.600
   Извлечено контекстов: 5
   Релевантных контекстов: 8

4. Вопрос: Каковы ключевые аспекты работы ML-разработчика?...
   Context Precision: 0.756
   Context Recall:    0.333
   Извлечено контекстов: 5
   Релевантных контекстов: 8

5. Вопрос: Что входит в сферу ответственности продуктового менеджера?...
   Context Precision: 0.833
   Context Recall: